# Tensorflow Own Object Detection Model in Google Colab
object Detection Sample images (After training complete. To get .pb file) [Part-2]

Created by Thyagarajan ( https://github.com/thyagarajank ) Project_File:https://github.com/thyagarajank/tensorflow-own-object-detection-model-in-google-colab

Note: 

**Go to runtime-->change runtime type--> GPU**   

(change to GPU if it is none)

* working on support for tensorboard

**Step-1** Connect Google Drive (for storage)

In [ ]:
# mount drive with colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step-2** TensorFlow Installation

In [ ]:
!pip install tensorflow-gpu==1.15.0

**Step-3** Download tensorflow model garden (model.zip)

In [ ]:
!wget https://github.com/tensorflow/models/archive/refs/heads/r1.13.0.zip

In [ ]:
!unzip r1.13.0.zip

In [ ]:
!mv models-r1.13.0 models-master

If Use my repository model files. Download it (**Optional**)

In [ ]:
!wget https://github.com/thyagarajank/tensorflow-own-object-detection-model-in-google-colab

In [ ]:
!unzip master.zip

In [ ]:
cp -a /model/. /content/models-master/research/object_detection/

**Step-4** Download model files

In [ ]:
%cd models-master/research/object_detection

Download Google sample the model (faster_rcnn_inception_v2_coco_2018_01_28)
Check it https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [ ]:
!tar -zxvf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [ ]:
%cd ..

/content/models-master/research


In [ ]:
!pwd

**Step-5** Tensor & Python Installation

In [ ]:
!python setup.py build
!python setup.py install

In [ ]:
!pwd

**Step-6** Dependency tools installation

In [ ]:
!pip install pycocotools

In [ ]:
!protoc --python_out=. ./object_detection/protos/image_resizer.proto

In [ ]:
%%bash
%cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
%cd models/research
pip install .

In [ ]:
pip install Cartopy==0.17.0

In [ ]:
!pip install -q matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
import libarchive
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot
!apt-get -qq install python-cartopy python3-cartopy
import cartopy

In [ ]:
%cd slim

In [ ]:
!python setup.py build
!python setup.py install

In [ ]:
!pwd
%cd ..
%cd object_detection/

**Step-7** Upload Training dataset,tf.record files and config files.

Upload your dataset, tf.record and config zip files in your google drive (Give the file path in copy command 'cp-r' filepath )

In [ ]:
!cp -r /content/drive/'My Drive'/filename.zip /content/models-master/research/object_detection/
!unzip rccn.zip

In [ ]:
cp -a /content/models-master/research/object_detection/filename/. /content/models-master/research/object_detection/

**Step-8** final setup

In [ ]:
%cd ..
!pwd

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
%cd object_detection/

/content/models-master/research/object_detection


**Step-9** Object detection program

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from utils import label_map_util

from utils import visualization_utils as vis_util
#  model Name.
MODEL_NAME = 'inference_graph'
#MODEL_FILE = MODEL_NAME + '.tar.gz'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'labelmap.pbtxt')
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.

# OWN IMAGE FILE PATH
PATH_TO_TEST_IMAGES_DIR = '/content/drive/MyDrive/sample_images'
#RANGE
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 4) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (24, 16)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  %matplotlib inline
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
  print('Done')
  plt.show()